In [50]:
import pandas as pd
import numpy as np
import random

In [3]:
data_from_scrapping = pd.read_csv('stat_questions.csv')
data_from_scrapping.head()

,web-scraper-order,web-scraper-start-url,questions_and_answers
0,1675270905-1,https://intellipaat.com/blog/interview-questio...,1. How is the statistical significance of an i...
1,1675270905-2,https://intellipaat.com/blog/interview-questio...,Hypothesis testing is used to find out the sta...
2,1675270905-3,https://intellipaat.com/blog/interview-questio...,"After calculating the p-value, the null hypoth..."
3,1675270905-4,https://intellipaat.com/blog/interview-questio...,NaN
4,1675270905-5,https://intellipaat.com/blog/interview-questio...,2. Where are long-tailed distributions used?


In [4]:
data_droped = data_from_scrapping.drop(['web-scraper-order', 'web-scraper-start-url'], axis=1).copy()
data_droped.head()

,questions_and_answers
0,1. How is the statistical significance of an i...
1,Hypothesis testing is used to find out the sta...
2,"After calculating the p-value, the null hypoth..."
3,NaN
4,2. Where are long-tailed distributions used?


In [7]:
data_droped.dropna(how='all', inplace=True)
data_droped.reset_index(drop=True, inplace=True)
data_droped_string = data_droped.convert_dtypes(convert_string=True)
data_droped_string.info()

# converting the column to array:
qa_array = data_droped_string['questions_and_answers'].to_numpy()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 272 entries, 0 to 271
Data columns (total 1 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   questions_and_answers  272 non-null    string
dtypes: string(1)
memory usage: 2.2 KB


In [32]:
rows_with_digits = []
rows_without_digits = []
answer_list = []
for i in qa_array:
    if not i[0].isdigit():
        rows_without_digits.append(i)
    else:
        # Joining the answers stored in list into single string and clearing the list, for next answers
        # matching the question
        if rows_without_digits:
            result_answer = ''.join(rows_without_digits)
            answer_list.append(result_answer)
            rows_without_digits = []
        rows_with_digits.append(i)
# Last if to make sure the last separated answer is also added to the list
if rows_without_digits:
    result_answer = ''.join(rows_without_digits)
    answer_list.append(result_answer)
        
#print(f'Rows with digits: {rows_with_digits}\n')
#print(f'List of answers: {answer_list}')

# Saving the results to the data frame with the questions in one column an the answers in another:
# Checking if the answers and question list are  matching in lenght:
try:
    if len(rows_with_digits) == len(answer_list):
        # creating the dict for pandas:
        qa_dict_to_df = {'questions': rows_with_digits, 'answers': answer_list}
except ValueError: 
    print("Lengths of provieded data frame differ in length, separation faild")

try:
    # creating df from dict
    df_separated_qa = pd.DataFrame(qa_dict_to_df)
except (TypeError, KeyError): 
    print("Creating of dataframe with answer, failed, check the dict used to creating the df, or the lists with answers")


In [34]:
df_separated_qa.tail()

,questions,answers
68,69. What is the impact of outliers in statistics?,Outliers in statistics have a very negative im...
69,"70. When creating a statistical model, how do ...",Overfitting can be detected by cross-validatio...
70,71. What is a survivorship bias?,The survivorship bias is the flaw of the sampl...
71,72. What is an undercoverage bias?,The undercoverage bias is a bias that occurs w...
72,74. What is the relationship between standard ...,Standard deviation is the square root of stand...


In [35]:
# Making codes for questions and answers:

def generate_code(row_index):
    return 'A' + str(row_index+1)

question_codes = []
answer_codes = []
for i, row in df_separated_qa.reset_index().iterrows():
    question_codes.append(generate_code(i))
    answer_codes.append(generate_code(i))
    
# Adding codes as columns to data frame:
df_separated_qa['question_codes'] = question_codes
df_separated_qa['answer_codes'] = answer_codes

In [57]:
df_separated_qa.to_csv('qa_with_codes.csv')

In [58]:
# Spliting questions to one dict with codes and answers to another:

question_dict = df_separated_qa.set_index('question_codes')['questions'].to_dict()
answer_dict = df_separated_qa.set_index('answer_codes')['answers'].to_dict()

In [49]:
# Making main loop for asking how many questions we want to answer, and to randomize the answers:
def get_and_check_the_input(max_number_of_questions):
    attempts = 0
    while attempts <=3:
        input_val = input(f'Please write number of question you want ot answer (max number: {max_number_of_questions}): ')
        if str(input_val).isdigit():
            if int(input_val) <= max_number_of_questions:
                return int(input_val)
            
        attempts +=1
        print("Invalid input try again")
    
    if attempts == 4:
        print("Program ge the wrong input 3 times, exiting the program")
        exit()


try:
    max_number_of_questions = len(question_dict)
    number_of_question_to_generate = 0
    
    try:
        number_of_question_to_generate = get_and_check_the_input(max_number_of_questions)
    except:
        print("Function for checking the input value not working properly.")
        
    if number_of_question_to_generate is not None:
        print(f"You want to genrate {number_of_question_to_generate} random questions.")
except:
    print('Something in the input went wrong.')

Invalid input try again
Invalid input try again
You want to genrate 20 random questions.


In [73]:

input_val = 3
answers_to_random_keys = {}
copy_of_answers_dict = answer_dict.copy()
random_question_keys = random.sample(list(question_dict.keys()), input_val)
#Getting proper answers and deleting this keys from random pool:
for key in random_question_keys:
    if key in answer_dict:
        answers_to_random_keys[key] = answer_dict[key]
        copy_of_answers_dict.pop(key)

answers_to_fullfil_options_keys = random.sample(list(copy_of_answers_dict.keys()), input_val*3)
answers_to_fullfil_rest_of_question_options = {}
#getting the rest of the answers to another dict:
for key in answers_to_fullfil_options_keys:
    if key in answer_dict:
        answers_to_fullfil_rest_of_question_options[key] = answer_dict[key]

# creating answers list of lists with dict keys:
answers_keys_list = []
# getting one correct answer key from proper answer dict:
for key in answers_to_random_keys:
    keys_from_fullfil_dict = random.sample(list(answers_to_fullfil_rest_of_question_options.keys()), 3)
    answers_keys_list.append([key] + keys_from_fullfil_dict)
    
# shuffling the inner lists
for inner_list in answers_keys_list:
    random.shuffle(inner_list)
#print(answers_keys_list)

# creating the questions with 4 answears, as question key: list of answers:
q_a_dict = {}
for key in random_question_keys:
    if key in question_dict:
        for i in answers_keys_list:
            if key in i:
                q_a_dict[key] = i


        

{'A60': ['A63', 'A60', 'A23', 'A36'],
 'A40': ['A52', 'A37', 'A20', 'A40'],
 'A73': ['A52', 'A11', 'A63', 'A73']}

In [74]:
# Creating the quiz:
score = 0

for question_key, answers_keys in q_a_dict.items():
    print(f'{question_dict[question_key]}')
    for i, answer in enumerate(answers_keys):
        print(f'{i+1}. {answer_dict[answer]} \n')
    user_input = input("Enter the number of your answer: ")
    if answers_keys[int(user_input)-1] == question_key:
        print("Correct! \n")
        score += 1
    else: 
        print('Incorect!')
        print(f'The correct answer is {answer_dict[question_key]} \n')

print(f'Youe got {score} out of {len(q_a_dict)} questions correct.')



60. What are the types of biases that you can encounter while sampling?
1. Hash tables are the data structures that are used to denote the representation of key-value pairs in a structured way. The hashing function is used by a hash table to compute an index that contains all of the details regarding the keys that are mapped to their associated values. 

2. There are three types of biases:Selection biasSurvivorship biasUnder coverage bias 

3. Mean is the average of a collection of values. We can calculate the mean by dividing the sum of all observations by the number of observations. 

4. True, a normal curve will have the area under unity and the symmetry around zero in any distribution. Here, all of the measures of central tendencies are equal to zero due to the symmetric nature of the standard normal curve. 

Correct!
40. What types of variables are used for Pearson’s correlation coefficient?
1. Following steps can be used to determine the length of sharks:Define the confidence lev